In [1]:
import networkx as nx 
import igraph as ig
import numpy as np 
import scipy as sp 
import pickle
import pandas as pd 
import collections
import requests 
import math 
import json
import re
from imblearn.combine import SMOTETomek

Constructing the graphs from the co-occurrence logs in CSV format

In [3]:
def make_nx_graph(path,timestamp,graph_format="gpickle",output_dir="graphs/"):
    graph_file = open(path,"rb")
    co_occurrences = graph_file.readlines()
    co_occurrences = [x.strip() for x in co_occurrences] 
    
    graph = nx.parse_edgelist(co_occurrences, nodetype = str, data=(('weight',int),), delimiter=",")
    
    outfile = output_dir+"graph_"+str(timestamp)+"."+graph_format
    if graph_format == "gpickle":
        nx.write_gpickle(graph,outfile)
    
    elif graph_format == "gexf":
        nx.write_gexf(graph,outfile)
        
    elif graph_format == "gml":
        nx.write_gml(graph,outfile)
        
    elif graph_format == "pajek":
        nx.write_pajek(graph,outfile)
    
    elif graph_format == "edgelist":
        nx.write_weighted_edgelist(graph,outfile)
    
    return {
        "outputfile":outfile,
        "graph":graph
    }

Community detection at each snapshot using iGraph. There is no interface between networkx and iGraph; hence, from the raw_data, the networkx-constructed graph is saved into pajek format to be used by iGraph in community detection.

In [4]:
def detect_communities(graph,timestamp,method="infomap",save_to_file=True,output_dir="graphs/",file_format="pajek"):
    if method == "infomap":
        results = graph.community_infomap(edge_weights="weight")
        
    elif method == "label_prop":
        results = graph.community_label_propagation(weights="weight")
        
    elif method == "multilevel":
        results = graph.community_multilevel(weights="weight")
        
    elif method == "walktrap":
        results = graph.community_walktrap(weights="weight")
    
    elif method == "louvain":
        results = graph.community_fastgreedy(weights="weight")
    
    graph.vs["community"] = results.membership
    
    if save_to_file: 
        graph.write_pajek(fname=output_dir + "graph_" + str(timestamp) + "_" + method + "_comms."+file_format)
    
    return graph

def get_assigned_community(node_id,graph):
    assigned_comm = graph.vs[node_id]["community"]
    return assigned_comm

def get_community(community_id,graph):
    nodes_in_comm = graph.vs.select(community_eq=community_id)
    return [v.index for v in nodes_in_comm]

def get_all_communities(graph,size=3):
    communities = set(graph.vs["community"])
    all_communities = {}
    for comm in communities:
        nodes_in_comm = graph.vs.select(community_eq=comm)
        if len(nodes_in_comm)>=size:
            all_communities[comm] = [v.index for v in nodes_in_comm]
    
    return all_communities

def get_vertex_names(nodes,graph):
    return [graph.vs[node]["id"] for node in nodes]

Utility functions for community matching and event detection across snapshots.
Theta is the similarity threshold for matching a pair of communities. 
Phi is th fluctuation threshold that measures the change in size of a community that has a match.

In [85]:
def compute_similarity(comm1,comm2): #based on (Hopcroft et al.,2004)
    common = set(comm1).intersection(set(comm2))
    return min(len(common)*1.0/len(comm1), len(common)*1.0/len(comm2))

def compute_fluctuation(comm1,comm2):
    return (len(comm2)*1.0/len(comm1))-1

def find_matches(graph1,graph2,theta=0.25,comm_size=3):
    
    communities1 = get_all_communities(graph1) #set of communities from snapshot 1
    communities2 = get_all_communities(graph2) #set of communities from snapshot 2
    
    print "There are %s communities/topics in snapshot i."%len(communities1)
    print "There are %s communities/topics in snapshot i+1."%len(communities2)
    
    matches = {} #contains the mapping of communities from snapshot 1 to snapshot 2
    
    for id1,nodes1 in communities1.iteritems():  #communities in time n
        similarities = dict()
        
        if len(nodes1)>= comm_size:
            nodes1 = get_vertex_names(nodes1,graph1)
            
            for id2,nodes2 in communities2.iteritems(): #communities in time n+1  
                if len(nodes2) >= comm_size:
                    nodes2 = get_vertex_names(nodes2,graph2)
                    theta_p = compute_similarity(nodes1,nodes2)  
                    similarities[id2]=theta_p
            
            non_zero = filter(lambda x: x != 0, similarities.values()) 
            match = [item for item in non_zero if item >= theta]

            if len(match) >= 1:  #match found 
                matched = similarities.keys()[similarities.values().index(match[0])] 
                matches[len(matches)+1] = {"1": id1,"2":matched}

            elif len(match) == 0: #match not found
                matches[len(matches)+1] = {"1":id1,"2":-1} 
    
    matches_df = pd.DataFrame.from_dict(matches,orient="index")
    return matches_df

def detect_events(graph1,graph2,matches_df,base_timestamp,save_to_file=True,output_dir="model/",phi=0.10):
    
    matches_df.columns = ["snap1","snap2"]
    
    events = {} #events/labels of communities from snapshot 1 based on their match in snapshot 2 
    #communities without a match are considered to have dissolved 
    dissolved = matches_df[matches_df["snap2"]== -1]
    
    for community1 in dissolved["snap1"]:
        events[community1] = "dissolve"
            
    #detect merging 
    matches_df["merges"] = matches_df.duplicated('snap2')
    merged =  matches_df[(matches_df["merges"] == True) & (matches_df["snap2"] != -1)]
    
    for community1 in merged["snap1"]:
        events[community1] = "merge"
    
    #detect splitting 
    matches_df["splits"] = matches_df.duplicated('snap1')
    split = matches_df[(matches_df["splits"] == True) & (matches_df["snap2"] !=-1)]
    
    for community1 in merged["snap2"]:
        events[community1] = "split"
    
    #detect growth, shrink, and survival based on fluctuation threshold phi 
    matches_df_persist = matches_df[(matches_df["merges"] == False) & (matches_df["snap2"] !=-1) & (matches_df["splits"]==False)]
    for index, row in matches_df_persist.iterrows():
        nodes1 = get_community(row["snap1"],graph1)
        nodes2 = get_community(row["snap2"],graph2) 
        
        fluctuation = compute_fluctuation(nodes1,nodes2)
        
        if fluctuation >= phi:
            events[row["snap1"]] = "growth"
        
        elif fluctuation <= -phi:
            events[row["snap1"]] = "shrink"
            
        else:
            events[row["snap1"]] = "survive"
    
    matches_df = matches_df[["snap1","snap2"]]
    events_df = pd.DataFrame.from_dict(events,orient="index")
    mapping = matches_df.join(events_df,on="snap1",how="inner")
    mapping.columns = ["snap1","snap2","event"]
    
    event_stats = mapping[["snap1","event"]].groupby(['event']).count()
    print event_stats
    
    if save_to_file:
        events_df.to_csv(output_dir+str(base_timestamp)+"_events.csv",index_label="id")
    
    return {
        "events" : events_df,
        "mapping": mapping,
        "event_stats": event_stats
    }

Utility functions for extracting features from communities in snapshots

In [99]:
def get_node_features(graph,timestamp,save_to_file=True,file_format="pajek",output_dir="graphs/"):
    graph.vs["pagerank"] = graph.pagerank(directed=False,weights='weight')
    
    degree_centrality = graph.degree(graph.vs)
    degree_centrality = (np.array(degree_centrality)*1.0)/np.sum(np.array(degree_centrality))
    graph.vs["degree"] = degree_centrality 
    
    clustering_coefficient = graph.transitivity_local_undirected(weights='weight')
    graph.vs["clustering_coefficient"] = clustering_coefficient 
    
    if save_to_file:
        graph.write_pajek(output_dir + "graph_" + str(timestamp) + "_node_atts." + file_format)
        
    return graph
    
def get_structural_features(communities,timestamp,graph,save_to_file=True,file_format="csv",output_dir="model/"):
    feats = {}
    
    for community_id,nodes in communities.iteritems():
        structural_feats = {}
        community_subgraph = graph.subgraph([node for node in nodes])

        inside_edges = graph.es.select(_within=nodes)
        inside_weights = [e["weight"] for e in inside_edges]

        outside_edges = []
        for node in nodes:
            #incident_to_node = [e.tuple for e in graph.es.select(_source=node)]
            incident_to_node = graph.es.select(_source=node)
            incident_to_node = list(filter(lambda x: x not in inside_edges, incident_to_node))
            outside_edges = outside_edges + incident_to_node

        outside_weights = [e["weight"] for e in outside_edges]

        structural_feats["size"] = len(nodes)
        structural_feats['inside_edges'] = len(inside_edges)
        structural_feats['inside_weights'] = np.sum(inside_weights)
        structural_feats['outside_edges'] = len(outside_edges)
        structural_feats['outside_weights'] = np.sum(outside_weights)
        structural_feats['weight_ratio'] = structural_feats['inside_weights'] * 1.0/structural_feats['outside_weights']
        total_comm_weight = structural_feats['inside_weights'] + structural_feats['outside_weights']
        structural_feats['inside_weight_ratio'] = structural_feats['inside_weights'] * 1.0 / total_comm_weight
        structural_feats['outside_weight_ratio'] = structural_feats['outside_weights'] * 1.0 /total_comm_weight 

        structural_feats['inside_weights_mean'] = np.mean(inside_weights)
        structural_feats['inside_weights_75'] = np.percentile(inside_weights,75)
        structural_feats['inside_weights_50'] = np.percentile(inside_weights,50)
        structural_feats['inside_weights_25'] = np.percentile(inside_weights,25)
        structural_feats['inside_weights_std'] = np.std(inside_weights)

        if len(outside_edges) > 0:
            cohesion = len(inside_edges)*1.0/len(outside_edges)
            structural_feats['cohesion'] = cohesion
            
        possible_edges = len(nodes)*((len(nodes)-1))
        non_edges = possible_edges - len(inside_edges)
        non_edges = non_edges*1.0/possible_edges
        structural_feats['unlinked'] = non_edges
                                         
        #node-specific properties
        pageranks = community_subgraph.vs["pagerank"]
        degree_centrality = community_subgraph.vs["degree"]  
        clustering_coefficient = community_subgraph.vs["clustering_coefficient"]
            
        pageranks = filter(lambda x: not np.isnan(x),pageranks)
        degree_centrality = filter(lambda x: not np.isnan(x),degree_centrality)
        clustering_coefficient = filter(lambda x: not np.isnan(x),degree_centrality)
        
        structural_feats['pageranks_mean'] = np.mean(pageranks)
        structural_feats['pageranks_75'] = np.percentile(pageranks,75)
        structural_feats['pageranks_50'] = np.percentile(pageranks,50)
        structural_feats['pageranks_25'] = np.percentile(pageranks,25)
        structural_feats['pageranks_std'] = np.std(pageranks)
        structural_feats['pageranks_max'] = np.max(pageranks)
                                         
        structural_feats['degree_mean'] = np.mean(degree_centrality)
        structural_feats['degree_75'] = np.percentile(degree_centrality,75)
        structural_feats['degree_50'] = np.percentile(degree_centrality,50)
        structural_feats['degree_25'] = np.percentile(degree_centrality,25)
        structural_feats['degree_std'] = np.std(degree_centrality)
        structural_feats['degree_max'] = np.max(degree_centrality)
            
        structural_feats['clustering_coeff_mean'] = np.mean(clustering_coefficient)
        structural_feats['clustering_coefficient_75'] = np.percentile(clustering_coefficient,75)
        structural_feats['clustering_coefficient_50'] = np.percentile(clustering_coefficient,50)
        structural_feats['clustering_coefficient_25'] = np.percentile(clustering_coefficient,25)
        structural_feats['clustering_coefficient_std'] = np.std(clustering_coefficient)
        structural_feats['clustering_coefficient_max'] = np.max(clustering_coefficient)
            
        structural_feats['density'] = community_subgraph.density()
        structural_feats['subgraph_clustering_coeff'] = community_subgraph.transitivity_avglocal_undirected()
            
        feats[community_id] = structural_feats
            
    #return dataframe of the structural features of communities
    feats = pd.DataFrame.from_dict(feats,orient="index")
    feats = feats[~feats.isin([np.nan, np.inf, -np.inf]).any(1)]
    feats = feats.dropna(axis=0,how='any')
    
    #normalize the data 
    feats_normed = feats
    feats_normed = feats_normed.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    
    if save_to_file:
        #print feats
        #print feats_normed
        feats.to_csv(output_dir + str(timestamp) + "_" + "structural_feats." + file_format,index_col="id")
        feats_normed.to_csv(output_dir + str(timestamp) + "_" + "structural_feats_normed." + file_format,index_col="id")
        
    return feats_normed


Training and testing the topic evolution prediction model. This is just a demo on a subset of the data originally used in the experiments.

In [8]:
graph_dir = "graphs/" #where the constructed graphs will be stored. The graphs are weighted and undirected.
data_dir = "data/"    #contains the pre-processed co-occurrence logs of format: keyword1,keyword2,frequency
model_dir = "model/"  #topic evolution model

In [107]:
#Two consecutive snapshots from the dynamic network
timestamp1 = 1
timestamp2 = 2

#Build the graph based on the co-occurrences within the duration of the snapshot in timestamp1
location1 = data_dir + "cooc_1.csv"
output1 = make_graph(location1,timestamp1,graph_format="pajek")
graph1 = output1["graph"]
path1 = output1["outputfile"]

#Detect communities from the snapshot in timestamp1
i_graph1 =  ig.Graph.Read_Pajek(path1)
i_graph_comms1 = detect_communities(i_graph1,timestamp)

#Build the graph based on the co-occurrences within the duration of the snapshot in timestamp2

location2 = data_dir + "cooc_2.csv"
output2 = make_graph(location2,timestamp2,graph_format="pajek")
graph2 = output2["graph"]
path2 = output2["outputfile"]

#Detect communities from the snapshot in timestamp2
i_graph2 =  ig.Graph.Read_Pajek(path2)
i_graph_comms2 = detect_communities(i_graph2,timestamp)

In [9]:
#Get the set of communities discovered for a pair of network snapshots
graph1 = ig.Graph.Read_Pickle(graph_dir+"graph_1_comms.pickle")
graph2 = ig.Graph.Read_Pickle(graph_dir+"graph_2_comms.pickle")

In [86]:
#Matching communities across snapshots 
matches = find_matches(graph1,graph2)
events = detect_events(graph1,graph2,matches,timestamp1)

There are 644 communities/topics in snapshot i.
There are 618 communities/topics in snapshot i+1.
          snap1
event          
dissolve    154
growth      160
merge        34
shrink      137
split        13
survive     146


In [443]:
#Extract features for each topic found in the snapshot at timestamp1
graph_with_new_atts = get_node_features(graph1,timestamp1)
graph_communities = get_all_communities(graph_with_new_atts)
features = get_structural_features(graph1_communities,timestamp1,graph_with_new_atts)
feature_names = list(features.columns)

In [88]:
#Prepare the dataset for training
events["events"].columns = ["event"]
training_data = features.join(events["events"],how="inner")
training_data = training_data.dropna(axis=0,how='any')

In [89]:
#Resampling the dataset 
X = training_data[feature_names]
y = training_data["event"]
print "Size of dataset :",len(X)

smote_tomek = SMOTETomek(random_state=1234)
X_resampled, y_resampled = smote_tomek.fit_sample(X,y)

print "Size of resampled dataset: ",len(X_resampled)

Size of dataset : 617
Size of resampled dataset:  804


Comparison of different models  

In [90]:
def calculate_cross_val(model,X,y):
    accuracy = cross_val_score(model,X,y,cv=skf,scoring='accuracy')
    precision = cross_val_score(model,X,y,cv=skf,scoring='precision_macro') 
    recall = cross_val_score(model,X,y,cv=skf,scoring='recall_macro') 
    f1 = cross_val_score(model,X,y,cv=skf,scoring='f1_macro') 
    
    return {
        'acc_mean': accuracy.mean(),
        'acc_std': accuracy.std() * 2,
        'prec_mean': precision.mean(), 
        'prec_std': precision.std() * 2, 
        'recall_mean': recall.mean(), 
        'recall_std': recall.std() * 2, 
        'f1_mean': f1.mean(), 
        'f1_std': f1.std() * 2
        }

In [91]:
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score, cross_val_predict 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib
matplotlib.use('agg')
from sklearn.metrics import accuracy_score
import pylab as plt

skf = StratifiedKFold(n_splits=10)
results = {}

#Random Forest Classifier 
rfc = RandomForestClassifier(n_estimators=100)
results["rf"] = calculate_cross_val(rfc,X_resampled,y_resampled)

#Support Vector Machine 
svc = SVC()
results["svm"] = calculate_cross_val(svc,X_resampled,y_resampled)

#K-Nearest Neighbors 
knn = KNeighborsClassifier()
results["knn"] = calculate_cross_val(knn,X_resampled,y_resampled)

#Logistic Regression
log_reg = LogisticRegression()
results["log_reg"] = calculate_cross_val(log_reg,X_resampled,y_resampled)

#Adaboost 
adaboost = AdaBoostClassifier()
results["adaboost"] = calculate_cross_val(adaboost,X_resampled,y_resampled)

results = pd.DataFrame.from_dict(results)
print results

             adaboost       knn   log_reg        rf       svm
acc_mean     0.308407  0.493273  0.357670  0.584998  0.298789
acc_std      0.096540  0.122992  0.108801  0.210882  0.056704
f1_mean      0.279933  0.411611  0.279007  0.512190  0.193944
f1_std       0.107162  0.143433  0.096148  0.194943  0.055675
prec_mean    0.349305  0.418711  0.267748  0.537409  0.225855
prec_std     0.198468  0.162770  0.073681  0.180314  0.106918
recall_mean  0.295985  0.459194  0.334610  0.568496  0.265710
recall_std   0.098579  0.124156  0.106901  0.182928  0.052581


Since Random Forest appears to outperform the other types of classifiers using its default number of trees = 10. 
We tune the number of estimators in this part. 

In [92]:
trees = [10,25,50,75,100]
results = {}

for n in trees:
    rfc = RandomForestClassifier(n_estimators=n)
    results["rfc_"+str(n)] = calculate_cross_val(rfc,X_resampled,y_resampled)

results = pd.DataFrame.from_dict(results)
print results 

               rfc_10   rfc_100    rfc_25    rfc_50    rfc_75
acc_mean     0.545652  0.592117  0.570974  0.587387  0.582337
acc_std      0.152354  0.193418  0.164159  0.172586  0.170033
f1_mean      0.484736  0.524987  0.530235  0.527946  0.509277
f1_std       0.187953  0.169748  0.157523  0.181822  0.187272
prec_mean    0.517702  0.575099  0.511584  0.529847  0.564934
prec_std     0.165409  0.203551  0.185800  0.184997  0.207904
recall_mean  0.527280  0.571625  0.532573  0.564417  0.552989
recall_std   0.178415  0.167604  0.185586  0.198131  0.180387


Fitting the final working model to be used for predicting the evolution of the topics found in snapshot i+1 in the next snapshot i+2

In [97]:
import time
timestamp = int(time.time())

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_resampled,y_resampled)

feature_importance = rfc.feature_importances_
print feature_importance 

rank = []
for name, importance in zip(feature_names, feature_importance):
    rank.append((name,importance))
    
rank = sorted(rank, key=lambda x: x[1], reverse=True)

print "Top 10 Features: "
print rank[:10]

#Save model 
filename = 'rf_model_'+str(timestamp)+'.model'
print "Model filename :" + filename
pickle.dump(rfc, open(model_dir+filename, 'wb'))

[0.02518524 0.02333412 0.02904226 0.02769149 0.0339971  0.03101564
 0.02488512 0.03136686 0.02284923 0.03250596 0.0271487  0.03048361
 0.03161845 0.02418723 0.02352247 0.02795342 0.02055665 0.02991715
 0.03161467 0.03377006 0.02752948 0.02804944 0.0267387  0.02577623
 0.02927143 0.03035316 0.03033205 0.03201439 0.02536415 0.03891476
 0.03620412 0.0311559  0.0312109  0.04443986]
Importance of Features: 
[('inside_weights_std', 0.04443985615977736), ('pageranks_25', 0.03891476121394247), ('unlinked', 0.03620412327870571), ('inside_weights', 0.03399710483928934), ('inside_weights_50', 0.03377006270854713), ('pageranks_50', 0.032505963596189395), ('cohesion', 0.03201439421394688), ('pageranks_75', 0.03161844565656897), ('inside_weight_ratio', 0.031614668681590075), ('size', 0.031366856591139206), ('inside_weights_mean', 0.031210895014868716), ('outside_weights', 0.031155896069826973), ('inside_weights_75', 0.03101563901550026), ('pageranks_mean', 0.03048360525566303), ('density', 0.0303531

In [ ]:
#Extract features for topic found in the snapshot at timestamp2
graph_with_new_atts = get_node_features(graph2,timestamp2)
graph_communities = get_all_communities(graph_with_new_atts)
X_test = get_structural_features(graph_communities,timestamp2,graph_with_new_atts)
X_test = X_test[feature_names] 

In [137]:
#Using the resulting model to predict the evolution of topics at timestamp2
filename = 'rf_model_1516164779.model'
loaded_model = pickle.load(open(model_dir+filename, 'rb'))
predictions = loaded_model.predict(X_test)
X_test["prediction"] = predictions

print "Prediction Stats:"
print X_test.groupby(['prediction'],axis=0)['prediction'].count() #outputs community_id, predicted topic evolution
pd.DataFrame(X_test).to_csv(model_dir+str(timestamp2)+"_predictions.csv")

Prediction Stats:
prediction
dissolve    144
growth      113
merge        35
shrink      111
split        42
survive     150
Name: prediction, dtype: int64


Exploring which topics will grow, shrink, survive?

In [122]:
#Parameters for the PubMed API:
user_num = "129"
user_token = "4n3-eeb91dbd494821a63a5c"

In [123]:
def get_label(cui):
    request_url = "http://havoc.appliedinformaticsinc.com/concepts/"+cui+"?sab=MSH&user="+user_num+"&token="+user_token    
    havoc_response = requests.get(request_url)
        
    try:
        parents_res = havoc_response.json()
        term = parents_res['terms'][0] 
        return term
        
    except (ValueError, IndexError):
        return ' '
    

In [156]:
def get_community_keywords(community_nodes,graph):
    
    community_subgraph = graph.subgraph([node for node in community_nodes])
    pageranks = sorted(community_subgraph.vs["pagerank"],reverse=True)
    top_rank = pageranks[:10]
    cuis = []
    
    for rank in top_rank:
        cui = community_subgraph.vs.select(pagerank_eq=rank)["id"][0]
        cuis.append(cui)
        
    keywords = map(lambda x: get_label(x),cuis)
    return list(keywords) 

In [158]:
event = "growth" #replace with any event of interest 

#Sample topics that will grow
growing_topics = X_test[X_test["prediction"]==event].head()

for index, row in growing_topics.iterrows():
    community_nodes = get_community(index,graph2)
    print "Label of community index ",index
    print get_community_keywords(community_nodes,graph2)


Label of community index  0
[u'Escherichia coli', u'Genes', u'DNA', u'Genes, Bacterial', u'Mutation', u'Transcription, Genetic', u'Gene Expression Regulation', u'Bacterial Proteins', u'RNA, Messenger', u'Saccharomyces cerevisiae']
Label of community index  1
[u'Occupational activity of managing finances', u'Hospital Departments', u'Personnel Management', u'Hospital Administration', u'Hospitals', u'Medicare', u'Nursing Staff, Hospital', u'Nursing Homes', u'Health Insurance', u'Discipline of Nursing']
Label of community index  2
[u'Brain', u'Neurons', u'Aging', u'Spinal Cord', u'Cerebral cortex', u'Norepinephrine', u'Dopamine', u'Serotonin', u'Retina', ' ']
Label of community index  3
[u'Monoclonal Antibodies', u'T-Lymphocyte', u'Lymphocyte', u'macrophage', u'neutrophil', u'B-Lymphocytes', u'Lymphocyte Activation', u'Immunoglobulin G', u'Surface Antigens', u'Autoantibodies']
Label of community index  4
[u'Hypertensive disease', u'Kidney', u'Coronary heart disease', u'Myocardium', u'Heart